### Load TXT File

In [2]:
from langchain.document_loaders import TextLoader

In [3]:
loader = TextLoader("RCBAuction.txt")
data = loader.load()

In [9]:
data
data[0].page_content
#data[0].metadata  # {'source': 'RCBAuction.txt'}
#data[0]  # It stores in a array form

'Another auction is here, and RCB steps in with the same hopeâ€”to build a well-rounded team capable of clinching their first-ever IPL title. It has been 17 long years, and the Royal Challengers Bangaloreâ€™s quest for the coveted trophy continues. With renewed energy and optimism, RCB must craft a better strategy, as the foundation for winning an IPL trophy is often laid at the auction table. Time and again, the franchiseâ€™s loyal fans and supporters have been left disappointed by questionable decisions and picks during previous auctions. This time, the focus will be on avoiding past mistakes and assembling a balanced squad for IPL 2025. The first step towards this goal was evident in the retentions announced during the IPL 2025 retention window.\n\nadvertisement\n\nRCB splurged Rs 37 crore to retain three players, with Virat Kohli being their top retention for RS 21 crore. Rajat Patidar followed at Rs 11 crore, while Yash Dayal, retained under the uncapped player category, was value

### You can also load CSV File 

In [16]:
from langchain.document_loaders.csv_loader import CSVLoader

In [20]:
loader = CSVLoader("movies.csv")
data  = loader.load()
data[0]

Document(metadata={'source': 'movies.csv', 'row': 0}, page_content='movie_id: 101\ntitle: K.G.F: Chapter 2\nindustry: Bollywood\nrelease_year: 2022\nimdb_rating: 8.4\nstudio: Hombale Films\nlanguage_id: 3\nbudget: 1\nrevenue: 12.5\nunit: Billions\ncurrency: INR')

In [1]:
from langchain.document_loaders import UnstructuredURLLoader

In [3]:
loader = UnstructuredURLLoader(urls=[
    "https://www.indiatoday.in/sports/story/ipl-auction-prediction-which-players-can-rcb-buy-kl-rahul-yuzvendra-chahal-bengaluru-purse-2636711-2024-11-21",
    "https://www.indiatoday.in/india/video/maharashtra-election-results-exit-poll-forecast-2636911-2024-11-21?utm_source=Taboola&utm_medium=organic&utm_campaign=recirculation"
])

In [4]:
data = loader.load()
len(data)

2

In [6]:
#data[0]

In [7]:
data[0].metadata

{'source': 'https://www.indiatoday.in/sports/story/ipl-auction-prediction-which-players-can-rcb-buy-kl-rahul-yuzvendra-chahal-bengaluru-purse-2636711-2024-11-21'}

### Text Splitters
**Why do we need text splitters in first place?**

LLM's have token limits. Hence we need to split the text which can be large into small chunks so that each chunk size is under the token limit. There are various text splitter classes in langchain that allows us to do this.


In [8]:

# Taking some random text from wikipedia

text = """Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. 
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine. 
Set in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007 and was originally set to be directed by Steven Spielberg. 
Kip Thorne, a Caltech theoretical physicist and 2017 Nobel laureate in Physics,[4] was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar. 
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm. Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles. 
Interstellar uses extensive practical and miniature effects, and the company Double Negative created additional digital effects.

Interstellar premiered in Los Angeles on October 26, 2014. In the United States, it was first released on film stock, expanding to venues using digital projectors. The film received generally positive reviews from critics and grossed over $677 million worldwide ($715 million after subsequent re-releases), making it the tenth-highest-grossing film of 2014. 
It has been praised by astronomers for its scientific accuracy and portrayal of theoretical astrophysics.[5][6][7] Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades."""

#### Manual approach of splitting the text into chunks
 Say LLM token limit is 100, in that case we can do simple thing such as this

In [9]:
text[0:100]
# Well but we want complete words and want to do this for entire text, may be we can use Python's split funciton

'Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher N'

In [10]:
words = text.split(" ")
len(words)
chunks = []

In [11]:
s = ""
for word in words:
    s += word + " "
    if len(s)>200:
        chunks.append(s)
        s = ""
        
chunks.append(s)

In [12]:
chunks[:2]

['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. \nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt ',
 'Damon, and Michael Caine. \nSet in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in ']

Splitting data into chunks can be done in native python but it is a tidious process. Also if necessary, you may need to experiment with various delimiters in an iterative manner to ensure that each chunk does not exceed the token length limit of the respective LLM.

Langchain provides a better way through text splitter classes.

### Using Text Splitter Classes from Langchain
### CharacterTextSplitter

In [13]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size=200,
    chunk_overlap=0
)

In [14]:
chunks = splitter.split_text(text)
len(chunks)

Created a chunk of size 210, which is longer than the specified 200
Created a chunk of size 208, which is longer than the specified 200
Created a chunk of size 358, which is longer than the specified 200


9

In [15]:
for chunk in chunks:
    print(len(chunk))

105
120
210
181
197
207
128
357
253


As you can see, all though we gave 200 as a chunk size since the split was based on \n, it ended up creating chunks that are bigger than size 200.

Another class from Langchain can be used to recursively split the text based on a list of separators. This class is RecursiveTextSplitter. Let's see how it works


### RecursiveTextSplitter

In [16]:
text

'Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. \nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine. \nSet in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.\n\nBrothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007 and was originally set to be directed by Steven Spielberg. \nKip Thorne, a Caltech theoretical physicist and 2017 Nobel laureate in Physics,[4] was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar. \nCinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm. Principal photography began in late 2013 and took place 

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " "],  # List of separators based on requirement (defaults to ["\n\n", "\n", " "])
    chunk_size = 200,  # size of each chunk created
    chunk_overlap  = 0,  # size of  overlap between chunks in order to maintain the context
    length_function = len  # Function to calculate size, currently we are using "len" which denotes length of string however you can pass any token counter)
)

In [18]:
chunks = r_splitter.split_text(text)

for chunk in chunks:
    print(len(chunk))

105
120
199
10
181
197
198
8
128
191
165
198
54


In [20]:
len(chunks)

13

Let's understand how exactly it formed these chunks


In [21]:
first_split = text.split("\n\n")[0]
first_split

'Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. \nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine. \nSet in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.'

In [22]:
len(first_split)

439

Recursive text splitter uses a list of separators, i.e. separators = ["\n\n", "\n", "."]

So now it will first split using \n\n and then if the resulting chunk size is greater than the chunk_size parameter which is 200 in our case, then it will use the next separator which is \n

In [23]:
second_split = first_split.split("\n")
second_split

['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. ',
 'It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine. ',
 'Set in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.']

In [24]:
for split in second_split:
    print(len(split))

106
121
210


Third split exceeds chunk size 200. Now it will further try to split that using the third separator which is ' ' (space)

In [25]:
second_split[2]

'Set in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.'